In [49]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px

mapbox_token = 'pk.eyJ1IjoiYW5kcmVhc29zdGVkIiwiYSI6ImNsbmJxMjFndDA4dm8ybXJrMzhia2NqdnoifQ.fXDNIJ1LelhA1ypNiaJE9w'

df = pd.read_csv('Data/homicide_data.csv')
df['victim_full_name'] = df['victim_first'] +  ' ' + df['victim_last']

df = df.sort_values('state')

In [50]:
df = pd.read_csv('Data/homicide_data.csv')
df['victim_full_name'] = df['victim_first'] +  ' ' + df['victim_last']

df = df.sort_values('city')

In [51]:
filtered_df = df[df['state'] == 'DC']
np.min(filtered_df.lon), np.max(filtered_df.lon)

(-77.095084, -76.9119737)

In [52]:
def make_map(city):
    filtered_df = df[df['city'] == city] if city != 'all cities' else df

  
    fig = px.scatter_mapbox(
        filtered_df,
        lat="lat",
        lon="lon",
        hover_name="victim_full_name",
        hover_data=["victim_race", "victim_age", "victim_sex"],
        color = "disposition",
        height=700
    )
    
    title = f'Murders in {city}' if city is not None else ''

    
    fig.update_layout(mapbox_style="light", 
                      mapbox_accesstoken=mapbox_token, 
                      title = title,
                      #mapbox_center={"lat": center_lat, "lon": center_lon},  
                      mapbox_zoom = 15,
                      autosize = True
                      )
    
    fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
    fig.update_layout(clickmode='event+select',
                      hovermode='closest')


    
    return fig

app = dash.Dash(__name__)


cityoptions=[{'label': 'Select all', 'value': 'all cities'}]+[{'label': city, 'value': city} for city in sorted(df['city'].unique())]
# Define layout
app.layout = html.Div([
    dcc.Dropdown(
        id='state-dropdown',
        options=cityoptions,
        #value=df['city'].unique()[0],  # Set the default value
        value = None,
        placeholder = "Select city",
        style={'width': '200px'}
    ),
    dcc.Graph(id='murder-map')
])

selected_points = {'Closed without arrest': [], 'Open/No arrest': [], 'Closed by arrest': []}

# Define callback to update the map when dropdown selection changes
@app.callback(
    Output('murder-map', 'figure'),
    [Input('state-dropdown', 'value'),
     Input('murder-map', 'selectedData')]  
)
def update_map(selected_city, selected_data):
    fig = make_map(selected_city)
    

    if selected_data:
        latitudes = [point['lat'] for point in selected_data['points']]
        longitudes = [point['lon'] for point in selected_data['points']]
        print(selected_data['points'])
        center = {
            "lat": np.mean(latitudes),
            "lon": np.mean(longitudes)
        }
        zoom = 12  # Set your desired zoom level
        
        fig.update_layout(mapbox_center=center, mapbox_zoom=zoom)

        #selected_indices = [point['pointIndex'] for point in selected_data['points']]
        selected_points = [(point['pointIndex'], point['curveNumber']) for point in selected_data['points']]
        unique_curves = range(len(fig.data))
        selected_dict = {c: [] for c in unique_curves}
        for p,c in selected_points:
            selected_dict[c].append(p)
        
        for u in unique_curves:
            fig.data[u].update(selectedpoints = selected_dict[u])


    return fig


# Define callback to update the map when dropdown selection changes
@app.callback(
    Output('murder-map', 'figure', allow_duplicate=True),
    [Input('state-dropdown', 'value')], 
    prevent_initial_call=True
)
def update_map_on_city_select(selected_state):
    fig = make_map(selected_state)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)



[{'curveNumber': 0, 'pointNumber': 41, 'pointIndex': 41, 'lon': -106.614901, 'lat': 35.1194254, 'hovertext': 'ANNA VIGIL', 'customdata': ['White', '27', 'Female']}, {'curveNumber': 0, 'pointNumber': 48, 'pointIndex': 48, 'lon': -106.6230949, 'lat': 35.1119559, 'hovertext': 'PAUL AYALA', 'customdata': ['Hispanic', '28', 'Male']}, {'curveNumber': 1, 'pointNumber': 24, 'pointIndex': 24, 'lon': -106.6230526, 'lat': 35.1078262, 'hovertext': 'ROY CHAVEZ', 'customdata': ['Unknown', 'Unknown', 'Unknown']}, {'curveNumber': 2, 'pointNumber': 92, 'pointIndex': 92, 'lon': -106.6208873, 'lat': 35.1083875, 'hovertext': 'SAMIR AL ABBOUDY', 'customdata': ['Unknown', '44', 'Unknown']}, {'curveNumber': 2, 'pointNumber': 98, 'pointIndex': 98, 'lon': -106.6220024, 'lat': 35.1150547, 'hovertext': 'LEROY GARCIA', 'customdata': ['Unknown', '21', 'Unknown']}, {'curveNumber': 2, 'pointNumber': 231, 'pointIndex': 231, 'lon': -106.6199663, 'lat': 35.1095199, 'hovertext': 'RAYONO SMITH', 'customdata': ['Other', '